# Importing Dependencies

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys
import collections
from yacs.config import CfgNode as CN
import os
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
from torch.utils.data import Dataset
from torchvision import transforms, datasets
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision import transforms
from cityscapesscripts.helpers.labels import trainId2label as t2l
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from einops import rearrange
import shutil
import time
import tqdm
from csv import reader
from functools import reduce
import torchvision
import torch._utils
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import numpy as np
import math
from torchvision import models
import progressbar
from time import sleep
from progress.spinner import MoonSpinner
from pathlib import Path
import csv
import itertools
import random
import gc
import statistics
import pandas as pd
import logging
import functools
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# Defining Training devices

In [ ]:
device_rtx3090 = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_rtx3070 = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 8
NUM_EPOCHS = 500

# Directories

In [ ]:
data_dir = '/home/sourajit/DeepLearningProjects/TB/datasets'
MODEL_PATH_GC = '/home/sourajit/DeepLearningProjects/TB/datasets/Models/'+'modelGC.pth'
MODEL_PATH_G = '/home/sourajit/DeepLearningProjects/TB/datasets/Models/'+'modelG.pth'
MODEL_PATH_D = '/home/sourajit/DeepLearningProjects/TB/datasets/Models/'+'modelC.pth'

# Dataloader

In [ ]:
domains= ['china', 'india', 'usa', 'tbx']
domain_pairs = list(((src, tar) for src in domains for tar in domains if src != tar))

In [ ]:
domain_pairs

In [ ]:
class TBDataset(Dataset):
    def __init__(self, root_dir, label_type, distribution_type, transform=None):
        
        self.transform = transform
        self.label_type = label_type # 'classification_label' or 'domain_label'
        self.distribution_type = distribution_type # 'source' or 'target'
        
        self.folders_list = os.listdir(root_dir)
        self.Img_list = []
        for folder in self.folders_list:
            self.Img_list.extend(
                [root_dir+'/'+folder+'/'+path for path in os.listdir(root_dir+'/'+folder)]
            )
        
    def __len__(self):
        length = len(self.Img_list)
        return length
      
    def __getitem__(self, index):
        image = Image.open(self.Img_list[index])
        image = image.convert("L")
        label = None
        if (self.label_type == 'classification_label'):
            label = int(self.Img_list[index].split('/')[-2])
        elif (self.label_type == 'domain_label'):
            if (self.distribution_type == 'source'):
                label = 0
            elif (self.distribution_type == 'target'):
                label = 1
        if self.transform is not None:
            image = self.transform(image)
        x = transforms.ToTensor()(image)
        y = np.array(label)
        y = torch.from_numpy(y)
        y = y.type(torch.LongTensor)
        return x, y

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH))
])

In [ ]:
def create_subset(dataset, data_limit, num_classes=2):
    #print ("***1***", data_limit)
    data_indices = []
    per_class_data_limit = data_limit//num_classes
    target_counter = collections.Counter()
    with MoonSpinner('Data Loader Working...') as bar:
        for idx, data in enumerate(dataset):
            if (idx == len(dataset)-1):
                for i in range(data_limit - per_class_data_limit*num_classes):
                    print ("Aisi")
                    data_indices.append(idx)
            target = data[1].item()
            target_counter[target] += 1
            if target_counter[target] <= per_class_data_limit:
                data_indices.append(idx)
            bar.next()
    
    sub_dataset = torch.utils.data.Subset(dataset, data_indices)
    #print ("***2***", len(sub_dataset))
    return sub_dataset

In [ ]:
def stratify_split(dataset, split):
    #split is a number between 0 to 100 train:test=split:1
    train_indices = []
    test_indices = []
    target_stat = collections.Counter()
    with MoonSpinner('Data Loader Working...') as bar:
        for idx, data in enumerate(dataset):
            target = data[1].item()
            target_stat[target] += 1
        bar.next()
    for k in target_stat.keys():
         target_stat[k] = int(target_stat[k]*split/100.0)
    target_counter = collections.Counter()
    with MoonSpinner('Data Loader Working...') as bar:
        for idx, data in enumerate(dataset):
            target = data[1].item()
            target_counter[target] += 1
            if target_counter[target] <= target_stat[target]:
                train_indices.append(idx)
            else:
                test_indices.append(idx)
        bar.next()
    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    test_dataset = torch.utils.data.Subset(dataset, test_indices)
    return train_dataset, test_dataset

In [ ]:
def find_bigger_dataset(len_A, len_B):
    if (len_A >= len_B):
        return 2, int(len_A/len_B), len_A - len_B*int(len_A/len_B)
    else:
        return 1, int(len_B/len_A), len_B - len_A*int(len_B/len_A)

In [ ]:
def get_combined_TB_data(root_dir_src, root_dir_tar, 
                         transform=None, batch_size=1, split=80, 
                         shuffle=False, pin_memory=False):
            
    data_src_cls = TBDataset(root_dir_src, label_type='classification_label', 
                             distribution_type='None', transform=transform)
    data_src_dom = TBDataset(root_dir_src, label_type='domain_label', 
                             distribution_type='source', transform=transform)
    data_tar_dom = TBDataset(root_dir_tar, label_type='domain_label', 
                             distribution_type='target', transform=transform)
    dataset_idx, multiplier, remainder = find_bigger_dataset(len(data_src_dom), 
                                                  len(data_tar_dom))
    #print ("Before Balancing")
    #print ("dom_src, dom_tar: ", len(data_src_dom), len(data_tar_dom))
    if (dataset_idx == 1):
        temp = multiplier*[data_src_dom]
        subset = create_subset(data_src_dom, remainder, num_classes=1)
        temp.append(subset)
        data_src_dom = torch.utils.data.ConcatDataset(temp)
    elif (dataset_idx == 2):
        temp = multiplier*[data_tar_dom]
        subset = create_subset(data_tar_dom, remainder, num_classes=1)
        temp.append(subset)
        data_tar_dom = torch.utils.data.ConcatDataset(temp)
    #print ("After Balancing")
    #print ("dom_src, dom_tar: ", len(data_src_dom), len(data_tar_dom))
    combined_data_dom = torch.utils.data.ConcatDataset((data_src_dom, data_tar_dom))

    dataset_idx, multiplier, remainder = find_bigger_dataset(len(data_src_cls), 
                                                  len(combined_data_dom))
    #print ("Before Balancing")
    #print ("cls_src, dom_combined", len(data_src_cls), len(combined_data_dom))
    if (dataset_idx == 1):
        temp = multiplier*[data_src_cls]
        subset = create_subset(data_src_cls, remainder)
        temp.append(subset)
        data_src_cls = torch.utils.data.ConcatDataset(temp)
    elif (dataset_idx == 2):
        temp = multiplier*[combined_data_dom]
        subset = create_subset(combined_data_dom, remainder)
        temp.append(subset)
        combined_data_dom = torch.utils.data.ConcatDataset(temp)
    #print ("After Balancing")
    #print ("cls_src, dom_combined", len(data_src_cls), len(combined_data_dom))

    train_data_cls, test_data_cls = stratify_split(data_src_cls, split)
    train_loader_cls = torch.utils.data.DataLoader(train_data_cls, batch_size=batch_size, 
                                              shuffle=shuffle, pin_memory=pin_memory)
    test_loader_cls = torch.utils.data.DataLoader(test_data_cls, batch_size=batch_size, 
                                              shuffle=shuffle, pin_memory=pin_memory)

    train_data_dom, test_data_dom = stratify_split(combined_data_dom, split)
    train_loader_dom = torch.utils.data.DataLoader(train_data_dom, batch_size=batch_size, 
                                              shuffle=shuffle, pin_memory=pin_memory)
    test_loader_dom = torch.utils.data.DataLoader(test_data_dom, batch_size=batch_size, 
                                              shuffle=shuffle, pin_memory=pin_memory)

    return train_loader_cls, test_loader_cls, train_loader_dom, test_loader_dom

# Utility Functions

In [ ]:
def count_num_param(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
def print_param(num_param):
    num_param = str(num_param)
    ret = []
    length = len(num_param)
    for i in range(length):
        idx = i+1
        if (idx>1 and (idx-1)%3==0):
            ret.append(',')
        ret.append(num_param[length-idx])
    ret.reverse()
    ret = ''.join(ret)
    temp_a = '## Number of parameters: ' + ret  + ' ##'
    temp_b = '#' * len(temp_a)
    columns = shutil.get_terminal_size().columns
    print (f'{temp_b}'.center(columns))
    print (f'{temp_a}'.center(columns))
    print (f'{temp_b}'.center(columns))

# Model

In [ ]:
class Generator(torch.nn.Module):
    def __init__(self, num_of_filters):
        super(Generator, self).__init__()
        self.num_of_filters = num_of_filters
        self.Resnet50_model = models.resnet50(pretrained=False)
        self.Resnet50_model = nn.Sequential(*list(self.Resnet50_model.children())[:-2])
        self.Resnet50_model[0] = nn.Conv2d(1,64,7,2,3)
        self.Resnet50_model = nn.Sequential(*list(self.Resnet50_model.children()))
        
        self.A_UpConv_1 = nn.ConvTranspose2d(2048, 1024, (3,3), stride=(2,2))
        self.A_activation_1 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.A_UpConv_2 = nn.ConvTranspose2d(1024, self.num_of_filters, (3,3), 
                                             stride=(2,2))
        self.A_activation_2 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.A_conv = nn.Conv2d(self.num_of_filters, self.num_of_filters, 
                                kernel_size=3, bias=False)
        self.A_activation_3 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.A_maxpool = nn.MaxPool2d(3, stride=2)
        
        self.B_UpConv_1 = nn.ConvTranspose2d(self.num_of_filters, self.num_of_filters, 
                                             (3,3), stride=(2,2))
        self.B_activation_1 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.B_conv = nn.Conv2d(self.num_of_filters, self.num_of_filters, 
                                kernel_size=3, bias=False)
        self.B_activation_3 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        #self.B_maxpool = nn.MaxPool2d(3, stride=2)
        
        self.C_UpConv_1 = nn.ConvTranspose2d(self.num_of_filters, self.num_of_filters, 
                                             (3,3), stride=(2,2))
        self.C_activation_1 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        
    def forward(self, x):
        x = self.Resnet50_model(x)
        x = self.A_UpConv_1(x)
        x = self.A_activation_1(x)
        x = self.A_UpConv_2(x)
        x = self.A_activation_2(x)
        x = self.A_conv(x)
        x = self.A_activation_3(x)
        x = self.A_maxpool(x)
        x = self.B_UpConv_1(x)
        x = self.B_activation_1(x)
        x = self.B_conv(x)
        x = self.B_activation_3(x)
        #x = self.B_maxpool(x)
        x = self.C_UpConv_1(x)
        x = self.C_activation_1(x)
        return x

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self, num_of_filters):
        super(Discriminator, self).__init__()
        self.num_of_filters = num_of_filters
        self.vgg19_model = models.vgg19(pretrained=False)
        self.vgg19_model.features[0] = nn.Conv2d(512,64,3,1,1)
        self.vgg19_model = nn.Sequential(*list(self.vgg19_model.children())[:-2])
        self.Linear_1 = nn.Linear(self.num_of_filters, 256)
        self.Activation_1 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.Linear_2 = nn.Linear(256, 128)
        self.Activation_2 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.Linear_3 = nn.Linear(128, 64)
        self.Activation_3 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.Linear_final = nn.Linear(64, 2)
        self.Activation_final = nn.Sigmoid()
        
    def forward(self, x):
        x = self.vgg19_model(x)
        x = nn.Flatten()(x)
        x = self.Linear_1(x)
        x = self.Activation_1(x)
        x = self.Linear_2(x)
        x = self.Activation_2(x)
        x = self.Linear_3(x)
        x = self.Activation_3(x)
        x = self.Linear_final(x)
        x = self.Activation_final(x)
        return x

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self, num_of_filters):
        super(Classifier, self).__init__()
        self.num_of_filters = num_of_filters
        self.vgg19_model = models.vgg19(pretrained=False)
        self.vgg19_model.features[0] = nn.Conv2d(512,64,3,1,1)
        self.vgg19_model = nn.Sequential(*list(self.vgg19_model.children())[:-2])
        self.Linear_1 = nn.Linear(self.num_of_filters, 256)
        self.Activation_1 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.Linear_2 = nn.Linear(256, 128)
        self.Activation_2 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.Linear_3 = nn.Linear(128, 64)
        self.Activation_3 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=False)
        self.Linear_final = nn.Linear(64, 2)
        self.Activation_final = nn.Sigmoid()
        
    def forward(self, x):
        x = self.vgg19_model(x)
        x = nn.Flatten()(x)
        x = self.Linear_1(x)
        x = self.Activation_1(x)
        x = self.Linear_2(x)
        x = self.Activation_2(x)
        x = self.Linear_3(x)
        x = self.Activation_3(x)
        x = self.Linear_final(x)
        x = self.Activation_final(x)
        return x

In [ ]:
G = Generator(512)
#print_param(count_num_param(G))
D = Discriminator(512)
#print_param(count_num_param(D))
C = Classifier(512)
#print_param(count_num_param(C))

In [ ]:
criterion = nn.BCELoss()
Params_Classification = list(G.parameters()) + list(C.parameters())
Optimizer_Classification = optim.Adam(Params_Classification, 
                                      lr=0.0002, betas=(0.5, 0.999))
Params_DomainInvariance = list(G.parameters()) + list(D.parameters())
Optimizer_DomainInvariance_G = optim.Adam(G.parameters(), 
                                        lr=0.0002, betas=(0.5, 0.999))
Optimizer_DomainInvariance_D = optim.Adam(D.parameters(), 
                                        lr=0.0002, betas=(0.5, 0.999))

In [ ]:
def training(diseases_classification_trainset, domain_classification_trainset, 
            MODEL_PATH_GC, MODEL_PATH_G, MODEL_PATH_D, device, num_epochs):
    # 1st step: Train (DIF Generator -> disseases classifier)
    # 2nd step: Train (DIF Generator -> domain classifier)
    G.to(device)
    C.to(device)
    D.to(device)
    G.eval()
    C.eval()
    D.eval()
    for epoch in range(num_epochs):
        # Training The Classifier with (X_s, y_s) : Class Labels
        for batch_number, example in enumerate(diseases_classification_trainset):
            X_s, y_s = example
            X_s, y_s = X_s.to(device), y_s.to(device)
            G_X_s = G(X_s)
            y_hat_s = C(G_X_s)
            l_c = criterion(y_hat_s, y_s)
            Optimizer_Classification.zero_grad()
            l_c.backward()
            Optimizer_Classification.step()
        torch.save({
                'model_state_dict': model.state_dict(),
                'optim_state_dict': Optimizer_Classification.state_dict(),
                'epoch': epoch
        }, MODEL_PATH_GC)
        # Training The Domain Invariance Step with (X_s+t=X, d_s+t=d) : Domain Labels
        for batch_number, example in enumerate(domain_classification_trainset):
            X, d = example
            X, d = X.to(device), d.to(device)
            G_X = G(X)
            d_hat = D(G_X)
            l_d = criterion(d_hat, d)
            d_gen = torch.full(d.shape, 0.5)
            d_gen.to(device)
            l_g = criterion(d_hat, d_gen)
            
            Optimizer_DomainInvariance_G.zero_grad()
            Optimizer_DomainInvariance_D.zero_grad()
            l_d.backward()
            l_g.backward()
            Optimizer_DomainInvariance_G.step()
            Optimizer_DomainInvariance_D.step()
        torch.save({
                'model_state_dict': model.state_dict(),
                'optim_state_dict': Optimizer_DomainInvariance_G.state_dict(),
                'epoch': epoch
        }, MODEL_PATH_G)
        torch.save({
                'model_state_dict': model.state_dict(),
                'optim_state_dict': Optimizer_DomainInvariance_D.state_dict(),
                'epoch': epoch
        }, MODEL_PATH_D)
    return

# Training The model

In [ ]:
for i in range(len(domain_pairs)):
    src_dir = data_dir+'/'+domain_pairs[i][0]
    tar_dir = data_dir+'/'+domain_pairs[i][1]
    diseases_classification_trainset, diseases_classification_testset, \
    domain_classification_trainset, domain_classification_testset = \
    get_combined_TB_data(root_dir_src=src_dir, root_dir_tar=tar_dir, 
                         transform=transform, split=80, batch_size=BATCH_SIZE, 
                         shuffle=True, pin_memory=False)
    training(diseases_classification_trainset, domain_classification_trainset, 
            MODEL_PATH_GC, MODEL_PATH_G, MODEL_PATH_D, device, NUM_EPOCHS)